<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/Chile_2_Drug_Data_1_0_(Write).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Read into Chile 2 dataset and get its indexes

In [ ]:
import chardet
with open('CHLDiab2.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.73, 'encoding': 'ISO-8859-1', 'language': ''}

In [ ]:
import pandas as pd
CHLDiab2 = pd.read_csv('CHLDiab2.csv',encoding='ISO-8859-1')
CHLDiab2.head(1)

,Unnamed: 0,Codigo,Tipo de Adquisicion,Estado,Nombre Organismo,Region Unidad,Codigo Moneda,Fecha Creacion,Fecha Publicacion,Fecha Adjudicacion,...,Valor Total Ofertado,Cantidad Adjudicada,Monto Linea Adjudica,Fecha Envio Oferta,Oferta seleccionada,CHL-USD,Monto Unitario Oferta USD,Monto Linea Adjudica USD,Valor Total Ofertado USD,Tipo de Institucion
0,1,8636737,Licitación Pública Entre 100 y 1000 UTM (LE),8,I MUNICIPALIDAD DE CHIGUAYANTE,Región del Biobío,CLP,2020-01-07,2020-01-10,2020-01-24,...,516000.0,1200.0,516000.0,2020-01-15,Seleccionada,773.730435,0.555749,666.898931,666.898931,Municipalidad


In [ ]:
CHLDiab2DirtyName = CHLDiab2['Descripcion linea Adquisicion']

In [ ]:
len(CHLDiab2DirtyName) # length of drug names

5331

In [ ]:
CHLDiab2DirtyName = list(set(CHLDiab2DirtyName))
len(CHLDiab2DirtyName) # length of drug names after remove duplicates

952

In [ ]:
# uppercase everything
CHLDiab2DirtyNameU = []
for i in CHLDiab2DirtyName:
  CHLDiab2DirtyNameU.append(i.upper())

In [ ]:
CHLDiab2DirtyNameU[:5]

['GLIBENCLAMIDA 5 MG CM/CM REC (ITEM N° 71 DE LAS ESPECIFICACIONES TÉCNICAS).',
 'METFORMINA/GLIBENCLAMIDA 500/5 MG, CAJA DE 30 COMPRIMIDOS',
 'FORXIGA 10MG (DAPAGLIFLOZINA), 28 COMP. POR CAJA',
 'GLIBENCLAMIDA 4 MG. CAJA DE 1000 COMPRIMIDOS',
 'GLIBENCLAMIDA 5 MG X 60 COMPRIMIDOS']

In [ ]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = CHLDiab2DirtyNameU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 1.4 MB 9.3 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 65 kB 3.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=4dae9fd9c198fbc4ac13a48be5bb018e5c14b9d75e4154e67591ce2b121116c9
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
GLIBENCLAMIDE 5 MG CM/CM REC (ITEM N° 71 OF THE TECHNICAL SPECIFICATIONS).


In [ ]:
CHLDiab2DirtyNameU[0]

'GLIBENCLAMIDA 5 MG CM/CM REC (ITEM N° 71 DE LAS ESPECIFICACIONES TÉCNICAS).'

In [ ]:
CHLDiab2DirtyNameUEng = []
for item in CHLDiab2DirtyNameU:
  translation = translator.translate(item, dest='en')
  CHLDiab2DirtyNameUEng.append(translation.text)
# runs 1 mins

In [ ]:
CHLDiab2DirtyNameUEng[:5]

['GLIBENCLAMIDE 5 MG CM/CM REC (ITEM N° 71 OF THE TECHNICAL SPECIFICATIONS).',
 'METFORMIN/GLIBENCLAMIDE 500/5 MG, BOX OF 30 TABLETS',
 'FORXIGA 10MG (DAPAGLIFLOZIN), 28 TABLETS. PER BOX',
 'GLYBENCLAMIDE 4 MG. BOX OF 1000 TABLETS',
 'GLIBENCLAMIDE 5 MG X 60 TABLETS']

In [ ]:
# get the orignila index about medicine name for each drug name 

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

CHLDiab2DirtyNameUESpl = []
for item in CHLDiab2DirtyNameUEng:
  tokenlist = tokenizer.tokenize(item)
  CHLDiab2DirtyNameUESpl.append(tokenlist)

In [ ]:
CHLDiab2DirtyNameUESpl[:1]

[['GLIBENCLAMIDE',
  '5',
  'MG',
  'CM',
  '/',
  'CM',
  'REC',
  '(',
  'ITEM',
  'N',
  '°',
  '71',
  'OF',
  'THE',
  'TECHNICAL',
  'SPECIFICATIONS',
  ').']]

In [ ]:
CHLDiab2DirtyNameIndex  = []
for item in CHLDiab2DirtyNameUESpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  CHLDiab2DirtyNameIndex.append(templist)

In [ ]:
len(CHLDiab2DirtyNameIndex)

952

In [ ]:
CHLDiab2DirtyNameIndex[:5]

[['GLIBENCLAMIDE', 'TECHNICAL', 'SPECIFICATIONS'],
 ['METFORMIN', 'GLIBENCLAMIDE', 'TABLETS'],
 ['FORXIGA', 'DAPAGLIFLOZIN', 'TABLETS'],
 ['GLYBENCLAMIDE', 'TABLETS'],
 ['GLIBENCLAMIDE', 'TABLETS']]

In [ ]:
# delete the information in each piece of drug name after SIMILAR
def DelSIMILAR(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'SIMILAR':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
# delete the information in each piece of drug name after EQUIVALENT
def DelEQUIVALENT(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'EQUIVALENT':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
CHLDiab2DirtyNameIndexEngDelSIMILAR = []
for item in CHLDiab2DirtyNameIndex:
  add = DelSIMILAR(item)
  add = DelEQUIVALENT(add)
  CHLDiab2DirtyNameIndexEngDelSIMILAR.append(add)

In [ ]:
CHLDiab2DirtyNameIndexEngDelSIMILAR[:5]

[['GLIBENCLAMIDE', 'TECHNICAL', 'SPECIFICATIONS'],
 ['METFORMIN', 'GLIBENCLAMIDE', 'TABLETS'],
 ['FORXIGA', 'DAPAGLIFLOZIN', 'TABLETS'],
 ['GLYBENCLAMIDE', 'TABLETS'],
 ['GLIBENCLAMIDE', 'TABLETS']]

## Part 2: Set up indexes for clean catagol drug names

In [ ]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [ ]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [ ]:
CatalogNameUS[:5] # after split by ","

[['AFATINIB 40 MG', ' TABLET', ' TBD'],
 ['LOSARTAN 100 MG + HYDROCHLOROTHIAZIDE 25 MG', ' TABLET', ' TBD'],
 ['BLEOMYCIN 15 UI', ' POWDER FOR INJECTION', ' VIAL', ' 1X1'],
 ['FLUCONAZOLE 2 MG/ML', ' INJECTION', ' 100 ML VIAL', ' 10X1'],
 ['PROPOFOL 20 MG/ML', ' TBD', ' TBD']]

In [ ]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [ ]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [ ]:
PresentationForm[:5]# the presentation form

['\xa0TABLET',
 ' 50MG TABLET',
 ' POWDER FOR INJECTION',
 ' SOLUTION FOR INJECTION',
 ' TABLET (CHEWABLE)']

In [ ]:
PresentationDiction = []

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

for item in PresentationForm:
  tokenlist = tokenizer.tokenize(item)
  PresentationDiction.append(tokenlist)

In [ ]:
PresentationDiction2 = []

for item in PresentationDiction:
  for subitem in item:
    if len(subitem) > 3:
      PresentationDiction2.append(subitem)

In [ ]:
PresentationDiction2 = list(set(PresentationDiction2))

In [ ]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [ ]:
print(len(MainMedicine), MainMedicine[:5])

462 ['AFATINIB 40 MG', 'LOSARTAN 100 MG + HYDROCHLOROTHIAZIDE 25 MG', 'BLEOMYCIN 15 UI', 'FLUCONAZOLE 2 MG/ML', 'PROPOFOL 20 MG/ML']


In [ ]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [ ]:
MainMedicineSpl[:5]

[['AFATINIB', '40', 'MG'],
 ['LOSARTAN', '100', 'MG', '+', 'HYDROCHLOROTHIAZIDE', '25', 'MG'],
 ['BLEOMYCIN', '15', 'UI'],
 ['FLUCONAZOLE', '2', 'MG', '/', 'ML'],
 ['PROPOFOL', '20', 'MG', '/', 'ML']]

In [ ]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)

In [ ]:
MainMedicineIndex[:5]

[['AFATINIB'],
 ['LOSARTAN', 'HYDROCHLOROTHIAZIDE'],
 ['BLEOMYCIN'],
 ['FLUCONAZOLE'],
 ['PROPOFOL']]

In [ ]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [ ]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [ ]:
len(CatalogIndexData_df)

462

In [ ]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"AFATINIB 40 MG, TABLET, TBD",[AFATINIB]
1,"LOSARTAN 100 MG + HYDROCHLOROTHIAZIDE 25 MG, T...","[LOSARTAN, HYDROCHLOROTHIAZIDE]"
2,"BLEOMYCIN 15 UI, POWDER FOR INJECTION, VIAL, 1X1",[BLEOMYCIN]
3,"FLUCONAZOLE 2 MG/ML, INJECTION, 100 ML VIAL, 10X1",[FLUCONAZOLE]
4,"PROPOFOL 20 MG/ML, TBD, TBD",[PROPOFOL]


In [ ]:
# create a list for main medicine from catalog name data set
CatalogMedName = []
for item in MainMedicineIndex:
  for subitem in item:
    CatalogMedName.append(subitem)

In [ ]:
CatalogMedName = list(set(CatalogMedName))
len(CatalogMedName)

206

In [ ]:
CatalogMedName[:3]

['METHYLDOPA', 'CALCIUM', 'MILTEFOSINE']

## Part 3: Write the Chile 2 Index Data Set According to the Catalog Name Index

In [ ]:
# for the vocabulaies which are included in the medicine index get from clean data set, keep them in the 
# Chile 2 data index
def KeepMetAnd(subitem):
  addlist = []
  for element in subitem:
    if element in CatalogMedName:
      addlist.append(element)
  return addlist

In [ ]:
CHI2finIndex = []
for sublist in CHLDiab2DirtyNameIndexEngDelSIMILAR:
  add = KeepMetAnd(sublist)
  CHI2finIndex.append(add)

In [ ]:
# count the number of Chile index sub list which is null []
count = 0
for i in CHI2finIndex:
  if not i:
    count += 1

In [ ]:
print(len(CHI2finIndex), count) # the total number of drug names and the names with null index

952 431


In [ ]:
nullindexlist = []
for i in range(0, len(CHI2finIndex)):
  if not CHI2finIndex[i]:
    nullindexlist.append(i)

In [ ]:
!pip install -U spacy
!pip install scispacy

In [ ]:
## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

In [ ]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [ ]:
#Load specific model and pass text through
nlp = en_core_sci_lg.load()

In [ ]:
CHLDiab2DirtyNameIndexEngDelSIMILAR[:100]

[['GLIBENCLAMIDE', 'TECHNICAL', 'SPECIFICATIONS'],
 ['METFORMIN', 'GLIBENCLAMIDE', 'TABLETS'],
 ['FORXIGA', 'DAPAGLIFLOZIN', 'TABLETS'],
 ['GLYBENCLAMIDE', 'TABLETS'],
 ['GLIBENCLAMIDE', 'TABLETS'],
 ['HYPOGLUCIN', 'METFORMIN', '500MG', 'GLYBENCLAMIDE', 'TABLETS'],
 ['GLIBENCLAMIDA'],
 ['VILDAGLIPTINA', 'METFORMINA', 'COMPRIMIDOS'],
 ['DAPAGLIFLOZIN', 'METFORMIN', 'EXTENDED', 'RELEASE', 'TABLETS'],
 ['GLIBENCLAMIDA'],
 ['METFORMIN', 'GLIBENCLAMIDE', 'TABLET'],
 ['VILDAGLIPTIN',
  'METFORMIN',
  'HYDROCHLORIDE',
  'VILDAGLIPTIN',
  'METFORMIN',
  'HYDROCHLORIDE'],
 ['VILDAGLIPTINA', 'JALRA', 'GALVUS'],
 ['VILDAGLIPTINA', 'METFORMINA'],
 ['BOXES', 'METFORMIN', 'TABLETS', 'VILDAGLIPTIN', 'SUGGESTED', 'GALVUS'],
 ['FORXIGA'],
 ['VILDAGLIPTINE',
  'METFORMIN',
  'COMMERCIAL',
  'COATED',
  'TABLETS',
  'EXPIRATION',
  'EQUAL',
  'LONGER',
  'MONTHS',
  'ENTER',
  'INFORMATION',
  'INDICATING',
  'MONTH'],
 ['METFORMIN', 'VILDAGLIPTIN', 'TABLETS'],
 ['METFORMIN', 'GLIBENCLAMIDE', 'TABLETS'],

In [ ]:
CHI2finIndex[:100]

[['GLIBENCLAMIDE'],
 ['METFORMIN', 'GLIBENCLAMIDE'],
 ['DAPAGLIFLOZIN'],
 [],
 ['GLIBENCLAMIDE'],
 ['METFORMIN'],
 [],
 [],
 ['DAPAGLIFLOZIN', 'METFORMIN'],
 [],
 ['METFORMIN', 'GLIBENCLAMIDE'],
 ['VILDAGLIPTIN',
  'METFORMIN',
  'HYDROCHLORIDE',
  'VILDAGLIPTIN',
  'METFORMIN',
  'HYDROCHLORIDE'],
 [],
 [],
 ['METFORMIN', 'VILDAGLIPTIN'],
 [],
 ['METFORMIN'],
 ['METFORMIN', 'VILDAGLIPTIN'],
 ['METFORMIN', 'GLIBENCLAMIDE'],
 ['GLIBENCLAMIDE'],
 [],
 [],
 ['VILDAGLIPTIN'],
 [],
 ['METFORMIN', 'GLIBENCLAMIDE'],
 [],
 [],
 ['METFORMIN', 'VILDAGLIPTIN'],
 ['VILDAGLIPTIN', 'METFORMIN'],
 [],
 ['VILDAGLIPTIN', 'METFORMIN'],
 [],
 [],
 ['GLIBENCLAMIDE'],
 ['METFORMIN'],
 ['VILDAGLIPTIN'],
 ['VILDAGLIPTIN', 'METFORMIN', 'HYDROCHLORIDE'],
 ['METFORMIN', 'VILDAGLIPTIN'],
 ['GLIBENCLAMIDE'],
 ['VILDAGLIPTIN'],
 [],
 [],
 ['VILDAGLIPTIN', 'METFORMIN'],
 [],
 ['DAPAGLIFLOZIN'],
 ['GLIBENCLAMIDE'],
 [],
 ['METFORMIN', 'HYDROCHLORIDE'],
 ['METFORMIN', 'VILDAGLIPTIN'],
 [],
 [],
 [],
 ['METFORMIN'],
 

In [ ]:
Chile2drugOutDataName = []
for nullindex in nullindexlist:
  tempChiledrugname = CHLDiab2DirtyNameIndexEngDelSIMILAR[nullindex]
  tempChiledrugname_str = str(' '.join(tempChiledrugname))
  doc = nlp(tempChiledrugname_str)
  temptext = doc.text
  Chile2drugOutDataName.append(temptext)

In [ ]:
Chile2drugOutDataNameNullNumber = 0

for i in Chile2drugOutDataName:
  if not i:
    Chile2drugOutDataNameNullNumber += 1

In [ ]:
Chile2drugOutDataNameNullNumber

2

In [ ]:
# for these null element in CHIfnIndex list, use Chile1drugOutDataName to complete them
j = 0
for i in nullindexlist:
  CHI2finIndex[i] = Chile2drugOutDataName[j]
  j += 1

In [ ]:
for index in range(len(CHI2finIndex)):
  if not CHI2finIndex[index]:
    CHI2finIndex[index] = CHLDiab2DirtyNameIndexEngDelSIMILAR[index]

In [ ]:
nullindex = []
for index in range(len(CHI2finIndex)):
  if not CHI2finIndex[index]:
    nullindex.append(index)

In [ ]:
nullindex

[367, 658]

In [ ]:
Chile2PreList = []

for item in CHLDiab2DirtyNameIndexEngDelSIMILAR:
  addlist = []
  for subitem in item:
    if subitem in PresentationDiction2:
      addlist.append(subitem)
  Chile2PreList.append(list(set(addlist)))

In [ ]:
WeightList = []

for item in CHLDiab2DirtyNameUESpl:
  addlist = []
  for subitem in range(0, len(item)):
    if item[subitem].isdigit():
      if subitem + 1 < len(item):
        if item[subitem + 1].lower() in ['mg', 'ml']:
          addlist.append(item[subitem])
          addlist.append(item[subitem + 1])
  WeightList.append(list(set(addlist)))

In [ ]:
print(len(CHI2finIndex), len(Chile2PreList), len(WeightList))

952 952 952


In [ ]:
# for catalog name index dataframe, transform list to string
str_CHI2finIndex = []
for item in CHI2finIndex:
  if not isinstance(item, str):
    addsub = []
    for subitem in item:
      if subitem:
        addsub.append(subitem)
        newitem = ' '.join(addsub)
  else:
    newitem = item

  str_CHI2finIndex.append(newitem)

In [ ]:
str_CHI2finIndex[:4]


['GLIBENCLAMIDE',
 'METFORMIN GLIBENCLAMIDE',
 'DAPAGLIFLOZIN',
 'GLYBENCLAMIDE TABLETS']

In [ ]:
str_Chile2PreList = []

for item in Chile2PreList:
  newitem = ' '.join(item)
  str_Chile2PreList.append(newitem)

In [ ]:
str_WeightList = []

for item in WeightList:
  newitem = ' '.join(item)
  str_WeightList.append(newitem)

In [ ]:
Chile2DrugDataWrite = {"Chile Dirty Name English": CHLDiab2DirtyNameUEng, "Final Drug Index": str_CHI2finIndex, "Presentation Form": str_Chile2PreList, "Weight": str_WeightList}


In [ ]:
Chile2DrugDataWrite_df = pd.DataFrame(Chile2DrugDataWrite)

In [ ]:
Chile2DrugDataWrite_df


,Chile Dirty Name English,Final Drug Index,Presentation Form,Weight
0,GLIBENCLAMIDE 5 MG CM/CM REC (ITEM N° 71 OF TH...,GLIBENCLAMIDE,,MG 5
1,"METFORMIN/GLIBENCLAMIDE 500/5 MG, BOX OF 30 TA...",METFORMIN GLIBENCLAMIDE,,MG 5
2,"FORXIGA 10MG (DAPAGLIFLOZIN), 28 TABLETS. PER BOX",DAPAGLIFLOZIN,,
3,GLYBENCLAMIDE 4 MG. BOX OF 1000 TABLETS,GLYBENCLAMIDE TABLETS,,4 MG
4,GLIBENCLAMIDE 5 MG X 60 TABLETS,GLIBENCLAMIDE,,MG 5
...,...,...,...,...
947,VILDAGLIPTIN 50 MG X 28 TABLETS,VILDAGLIPTIN,,MG 50
948,GLYBENCLAMIDE 5 MG. CM. MATURITY OVER 12 MONTHS,GLYBENCLAMIDE MATURITY MONTHS,,MG 5
949,GALVUS 50 MG PACK OF 56 TABLETS,GALVUS TABLETS,,MG 50
950,METFORMIN 500 MG + GLIBENCLAMIDE 5 MG ( CONSUM...,METFORMIN GLIBENCLAMIDE,,MG 500 5


In [ ]:
Chile2DrugDataWrite_df.to_excel("Chile2DrugDataWrite.xlsx", sheet_name = 'Chile2DrugDataWrite')  